In [21]:
import os 
from glob import glob

In [28]:
def removefiles(files):
    for fi in files:
        os.remove(fi)

In [33]:
fpath = "/media/ljp238/12TBWolf/ARCHIEVE/FB_CHM/LMK/*.tif"
files = glob(fpath); print(len(files))
#removefiles(files)

84


In [41]:
m1files = [i for i in files if i.endswith('masked.tif')];print(len(m1files))
m2files = [i for i in files if 'masked.tif' not in i];print(len(m1files))

42
42


In [ ]:
def filelist2txt(filelist, txt):
    with open(txt, 'w') as f:
        for fi in filelist:
            f.write(fi+'\n')


def buildvrt(txt, epsg='4326'): # 4749
    vrt = txt.replace('.txt', '.vrt')
    cmd =  f'gdalbuildvrt -a_srs {epsg} -input_file_list {txt} {vrt}'
    os.system(cmd)


def filelist2vrt(filelist, txt):
    filelist2txt(filelist, txt)
    buildvrt(txt)


In [48]:
VRT_dpath = "/media/ljp238/12TBWolf/ARCHIEVE/ARCHIVE_VRT"

In [49]:
FB_CHM_masked_txt = os.path.join(VRT_dpath, 'FB_CHM_masked.txt')
FB_CHM_txt = os.path.join(VRT_dpath, 'FB_CHM.txt')

filelist2vrt(m1files, FB_CHM_masked_txt)
filelist2vrt(m2files, FB_CHM_txt)

0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
